###SETTING UP THE SQLite ENVIRONMENT

In [2]:
import sqlite3
import pandas as pd

# Load datasets
athletes = pd.read_csv("https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-07-27/athlete_events.csv")
regions = pd.read_csv("https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-07-27/noc_regions.csv")
sales = pd.read_csv("https://raw.githubusercontent.com/mwaskom/seaborn-data/master/tips.csv")
sales = pd.read_csv("https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv", sep='\t')


# Connect to SQLite in-memory DB
conn = sqlite3.connect(":memory:")

# Write DataFrames to SQL tables
athletes.to_sql("athletes_table", conn, index=False, if_exists="replace")
regions.to_sql("regions_table", conn, index=False, if_exists="replace")
sales.to_sql("sales_table", conn, index=False, if_exists="replace")

4622

###SECTION 1

The the top 5, total number of medals won by each country.

In [ ]:
pd.read_sql(
    """
    SELECT Team AS Country, COUNT(*) AS Total_Medals
    FROM athletes_table WHERE Medal <> "None"
    GROUP BY NOC ORDER BY Total_Medals DESC LIMIT 5;
    """, conn
)

,Country,Total_Medals
0,United States,5637
1,Soviet Union,2503
2,Germany,2165
3,Great Britain,2068
4,France,1777


The average age of athletes who won a Gold medal.

In [ ]:
pd.read_sql(
    """
    SELECT AVG(Age) FROM athletes_table WHERE Medal="Gold";
    """, conn
)

,AVG(Age)
0,25.901013


Number of distinct events in each sport.

In [ ]:
pd.read_sql(
    """
    SELECT Sport, COUNT(DISTINCT(Event)) FROM athletes_table GROUP BY Sport;
    """, conn
)

,Sport,COUNT(DISTINCT(Event))
0,Aeronautics,1
1,Alpine Skiing,10
2,Alpinism,1
3,Archery,29
4,Art Competitions,29
...,...,...
61,Tug-Of-War,1
62,Volleyball,2
63,Water Polo,2
64,Weightlifting,21


All athletes from the United States.

In [ ]:
pd.read_sql(
    """
    SELECT * FROM athletes_table WHERE NOC="USA";
    """, conn
)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,6,Per Knut Aaland,M,31.0,188.0,75.0,United States,USA,1992 Winter,1992,Winter,Albertville,Cross Country Skiing,Cross Country Skiing Men's 10 kilometres,None
1,6,Per Knut Aaland,M,31.0,188.0,75.0,United States,USA,1992 Winter,1992,Winter,Albertville,Cross Country Skiing,Cross Country Skiing Men's 50 kilometres,None
2,6,Per Knut Aaland,M,31.0,188.0,75.0,United States,USA,1992 Winter,1992,Winter,Albertville,Cross Country Skiing,Cross Country Skiing Men's 10/15 kilometres Pu...,None
3,6,Per Knut Aaland,M,31.0,188.0,75.0,United States,USA,1992 Winter,1992,Winter,Albertville,Cross Country Skiing,Cross Country Skiing Men's 4 x 10 kilometres R...,None
4,6,Per Knut Aaland,M,33.0,188.0,75.0,United States,USA,1994 Winter,1994,Winter,Lillehammer,Cross Country Skiing,Cross Country Skiing Men's 10 kilometres,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18848,135458,Rami Zur,M,27.0,175.0,77.0,United States,USA,2004 Summer,2004,Summer,Athina,Canoeing,"Canoeing Men's Kayak Doubles, 500 metres",None
18849,135458,Rami Zur,M,31.0,175.0,77.0,United States,USA,2008 Summer,2008,Summer,Beijing,Canoeing,"Canoeing Men's Kayak Singles, 500 metres",None
18850,135458,Rami Zur,M,31.0,175.0,77.0,United States,USA,2008 Summer,2008,Summer,Beijing,Canoeing,"Canoeing Men's Kayak Singles, 1,000 metres",None
18851,135543,"Victor Andrew ""Vic"" Zwolak",M,25.0,175.0,64.0,United States,USA,1964 Summer,1964,Summer,Tokyo,Athletics,"Athletics Men's 3,000 metres Steeplechase",None


Medals awarded each year.

In [ ]:
pd.read_sql(
    """
    SELECT Year, COUNT(*) AS Medals_Awarded
    FROM athletes_table WHERE Medal <> "None"
    GROUP BY Year;
    """, conn
)

,Year,Medals_Awarded
0,1896,143
1,1900,604
2,1904,486
3,1906,458
4,1908,831
5,1912,941
6,1920,1308
7,1924,962
8,1928,823
9,1932,739


All athlete records where height or weight is missing.

In [ ]:
pd.read_sql(
    """
    SELECT * FROM athletes_table WHERE Height IS NULL OR Weight IS NULL;
    """, conn
)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,None
1,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
2,8,"Cornelia ""Cor"" Aalten (-Strannood)",F,18.0,168.0,NaN,Netherlands,NED,1932 Summer,1932,Summer,Los Angeles,Athletics,Athletics Women's 100 metres,None
3,8,"Cornelia ""Cor"" Aalten (-Strannood)",F,18.0,168.0,NaN,Netherlands,NED,1932 Summer,1932,Summer,Los Angeles,Athletics,Athletics Women's 4 x 100 metres Relay,None
4,10,"Einar Ferdinand ""Einari"" Aalto",M,26.0,NaN,NaN,Finland,FIN,1952 Summer,1952,Summer,Helsinki,Swimming,Swimming Men's 400 metres Freestyle,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64258,135539,Marius Edmund Zwiller,M,18.0,NaN,NaN,France,FRA,1924 Summer,1924,Summer,Paris,Swimming,Swimming Men's 200 metres Breaststroke,None
64259,135542,Werner Zwingli,M,29.0,NaN,NaN,Switzerland,SUI,1956 Winter,1956,Winter,Cortina d'Ampezzo,Cross Country Skiing,Cross Country Skiing Men's 15 kilometres,None
64260,135542,Werner Zwingli,M,29.0,NaN,NaN,Switzerland,SUI,1956 Winter,1956,Winter,Cortina d'Ampezzo,Cross Country Skiing,Cross Country Skiing Men's 4 x 10 kilometres R...,None
64261,135552,Jan (Johann-) Zybert (Siebert-),M,20.0,NaN,NaN,Poland,POL,1928 Summer,1928,Summer,Amsterdam,Cycling,"Cycling Men's Team Pursuit, 4,000 metres",None


Replacing the missing height with the average athlete height.

In [ ]:
pd.read_sql(
    """
    SELECT Name, 	Height, COALESCE(Height, (SELECT AVG(Height) FROM athletes_table)) AS Coalesced_Height FROM athletes_table;
    """, conn
)

,Name,Height,Coalesced_Height
0,A Dijiang,180.0,180.00000
1,A Lamusi,170.0,170.00000
2,Gunnar Nielsen Aaby,NaN,175.33897
3,Edgar Lindenau Aabye,NaN,175.33897
4,Christine Jacoba Aaftink,185.0,185.00000
...,...,...,...
271111,Andrzej ya,179.0,179.00000
271112,Piotr ya,176.0,176.00000
271113,Piotr ya,176.0,176.00000
271114,Tomasz Ireneusz ya,185.0,185.00000


Total sales value per item.

In [ ]:
pd.read_sql(
    """
    SELECT item_name, SUM(CAST(REPLACE(item_price, '$', '') AS REAL)) AS total_sales_value
    FROM sales_table
    GROUP BY item_name;
    """, conn)

,item_name,total_sales_value
0,6 Pack Soft Drink,356.95
1,Barbacoa Bowl,672.36
2,Barbacoa Burrito,894.75
3,Barbacoa Crispy Tacos,120.21
4,Barbacoa Salad Bowl,106.40
5,Barbacoa Soft Tacos,250.46
6,Bottled Water,302.56
7,Bowl,29.60
8,Burrito,44.40
9,Canned Soda,137.34


Total unique customer orders (Assume each order_id is a customer.)

In [ ]:
pd.read_sql("SELECT  COUNT(DISTINCT order_id) AS Total_Customers FROM sales_table;",conn)

,Total_Customers
0,1834


###SECTION 2


Write one query for this section.

Find countries with high-performing athletes in the Olympics. Use at least JOIN, NESTED QUERY, CASE, and optionally WITH.

For each country:
- Count the number of athletes who won at least one medal.
- Determine the average age of those medalists.
- Create a new column called performance:
  - 'High' if average age is below 25
  - 'Medium' if between 25 and 30
  - 'Low' if above 30


In [4]:
pd.read_sql(
    """
     -- Create a temporary table called Medalists that contains distinct athletes who won medals
    WITH Medalists AS (
            SELECT DISTINCT NOC, ID, Age
            FROM athletes_table
            WHERE Medal <> 'None'
        )

        SELECT
            r.region AS Country,
            COUNT(DISTINCT m.ID) AS Number_of_Medalists,
            ROUND(AVG(m.Age), 2) AS Average_Age_of_Medalists,
             -- Categorize performance based on average age of medalists
            CASE
                WHEN AVG(m.Age) < 25 THEN 'High'
                WHEN AVG(m.Age) BETWEEN 25 AND 30 THEN 'Medium'
                ELSE 'Low'
            END AS Performance
        FROM Medalists m
        -- Join with regions table to get country names from NOC codes
        JOIN regions_table r ON m.NOC = r.NOC

        -- Group results by country
        GROUP BY r.region

        -- Only include countries that have at least one medalist
        HAVING COUNT(DISTINCT m.ID) > 0

        -- Order the results by:
        ORDER BY
            -- First by performance category (High first, then Medium, then Low)
            CASE Performance
                WHEN 'High' THEN 1
                WHEN 'Medium' THEN 2
                ELSE 3
            END,
            -- Then by number of medalists in descending order (most medalists first)
            Number_of_Medalists DESC;
       """, conn)

,Country,Number_of_Medalists,Average_Age_of_Medalists,Performance
0,China,670,23.36,High
1,Japan,653,24.44,High
2,South Korea,476,23.59,High
3,Romania,414,24.78,High
4,Bulgaria,280,24.78,High
...,...,...,...,...
131,United Arab Emirates,2,34.50,Low
132,Djibouti,1,31.00,Low
133,Iraq,1,NaN,Low
134,Kuwait,1,39.00,Low
